In [44]:
import pandas as pd
from types import GeneratorType
#Nested cross validation
from cv import NestedCV
from darts import TimeSeries
from darts.models import RNNModel,NBEATSModel,XGBModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape
from prophet import Prophet
import numpy as np
from datetime import datetime, timedelta

# Objectives
<h3>
1)Test cases to check the implementation of NestedCV <br/>
2)Build a timeseries model and use the cross validation technique for model evaluation
</h3>

In [45]:
# Creating sample dataset for testing NestedCV
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 2, 28)
date_range = pd.date_range(start_date, end_date, freq='D')

data = {
    'date': np.repeat(date_range, 3),  # Repeat each date three times for three groups
    'group_column': np.tile(['Group1', 'Group2', 'Group3'], len(date_range)),
    'value': np.random.randn(len(date_range) * 3)
}
data = pd.DataFrame(data)
data["date"] = pd.to_datetime(data["date"])

# nested cv
k = 3
cv = NestedCV(k)
splits = cv.split(data, "date")

# check return type
assert isinstance(splits, GeneratorType)

# check return types, shapes, and data leaks
count = 0
for train, validate in splits:
    
    # types
    assert isinstance(train, pd.DataFrame)
    assert isinstance(validate, pd.DataFrame)

    # shape
    assert train.shape[1] == validate.shape[1]

    # data leak
    assert train["date"].max() <= validate["date"].min()

    count += 1

# check number of splits returned
assert count == k

# We will be using the dataset provided in the assessment to construct basic time series models and validate our cross-validation logic.

In [25]:
data = pd.read_csv("train.csv")
data=data[~data["date"].isna()]

In [26]:
#this dataset contains monthly frequency.
pd.infer_freq(data['date'].unique())

'M'

<h2>We will be building a time series model to forecast demand for each brand</h2>

1)First, we will try simple machine learning models such as Facebook Prophet.  <br/>
2)Next, we will explore some deep learning models. 


In [27]:
#Number of unique Brands
data["brand"].unique()

array(['kinder-cola', 'adult-cola', 'orange-power', 'gazoza',
       'lemon-boost'], dtype=object)

In [31]:
# Check the number of data points, maximum date, and minimum date for each brand
for b in data["brand"].unique():
    print(b)
    temp=data[data["brand"]==b]
    print(f"Data points {len(temp)}")
    print(f"Start Date: {temp['date'].min()}")
    print(f"End Date: {temp['date'].max()}")

kinder-cola
Data points 1296
Start Date: 28/02/13
End Date: 31/12/17
adult-cola
Data points 1296
Start Date: 28/02/13
End Date: 31/12/17
orange-power
Data points 1296
Start Date: 28/02/13
End Date: 31/12/17
gazoza
Data points 1296
Start Date: 28/02/13
End Date: 31/12/17
lemon-boost
Data points 1296
Start Date: 28/02/13
End Date: 31/12/17


In [16]:
#There are no missing values in the dataset
data=data[["date","brand","quantity"]]
data['date'] = pd.to_datetime(data['date'], format='%d/%m/%y')
data.set_index('date', inplace=True)
#Resample the data to monthly frequency on the basis of brand
resampled_data=data.groupby(['brand', pd.Grouper(freq='M')]).sum().reset_index()

In [17]:
resampled_data

,brand,date,quantity
0,adult-cola,2012-01-31,315852.0
1,adult-cola,2012-02-29,232201.0
2,adult-cola,2012-03-31,359081.0
3,adult-cola,2012-04-30,444008.0
4,adult-cola,2012-05-31,469280.0
...,...,...,...
355,orange-power,2017-08-31,776399.0
356,orange-power,2017-09-30,646508.0
357,orange-power,2017-10-31,550004.0
358,orange-power,2017-11-30,434359.0


## Prophet

In [19]:
# Initialize Nested Cross-Validation with 5 folds
cv = NestedCV(5)
splits = cv.split(resampled_data, "date")
fold=1
final_result=pd.DataFrame({"Brand":[],"MAPE":[],"Fold":[]})

# Loop through each train-validation split in the cross-validation
for train, validate in splits:
    print(f"------ Fold {fold}--------")
    for brand, group in train.groupby('brand'): 
        temp=group[["date","quantity"]]
        temp.columns=["ds","y"]
        # Initialize and fit the Prophet model
        m=Prophet()
        m.fit(temp)
        # Prepare validation data for forecasting
        v_temp=validate[validate["brand"]==brand][["date","quantity"]]
        v_temp.columns=["ds","y"]
        # Generate forecast using the trained Prophet model
        forecast = m.predict(v_temp[["ds"]])
        # Merge actual and forecasted data
        result=pd.merge(v_temp, forecast[["ds","yhat"]], on='ds')
        # Calculate Absolute Percentage Error (APE) and Mean Absolute Percentage Error (MAPE)
        result['APE'] = np.abs((result['y'] - result['yhat']) / result['y']) * 100
        mape = np.mean(result['APE'])
        # print(f'MAPE of {brand}: {mape:.2f}%')
        final_result = pd.concat([final_result, pd.DataFrame([{"Brand": brand, "MAPE": mape, "Fold": fold}])], ignore_index=True)

    fold=fold+1 
    

01:40:36 - cmdstanpy - INFO - Chain [1] start processing
01:40:36 - cmdstanpy - INFO - Chain [1] done processing


------ Fold 1--------


01:40:36 - cmdstanpy - INFO - Chain [1] start processing
01:40:36 - cmdstanpy - INFO - Chain [1] done processing
01:40:36 - cmdstanpy - INFO - Chain [1] start processing
01:40:37 - cmdstanpy - INFO - Chain [1] done processing
01:40:37 - cmdstanpy - INFO - Chain [1] start processing
01:40:37 - cmdstanpy - INFO - Chain [1] done processing
01:40:37 - cmdstanpy - INFO - Chain [1] start processing
01:40:37 - cmdstanpy - INFO - Chain [1] done processing
01:40:37 - cmdstanpy - INFO - Chain [1] start processing
01:40:37 - cmdstanpy - INFO - Chain [1] done processing


------ Fold 2--------


01:40:37 - cmdstanpy - INFO - Chain [1] start processing
01:40:37 - cmdstanpy - INFO - Chain [1] done processing
01:40:37 - cmdstanpy - INFO - Chain [1] start processing
01:40:37 - cmdstanpy - INFO - Chain [1] done processing
01:40:37 - cmdstanpy - INFO - Chain [1] start processing
01:40:38 - cmdstanpy - INFO - Chain [1] done processing
01:40:38 - cmdstanpy - INFO - Chain [1] start processing
01:40:38 - cmdstanpy - INFO - Chain [1] done processing
01:40:38 - cmdstanpy - INFO - Chain [1] start processing
01:40:38 - cmdstanpy - INFO - Chain [1] done processing


...

01:40:38 - cmdstanpy - INFO - Chain [1] start processing
01:40:38 - cmdstanpy - INFO - Chain [1] done processing
01:40:38 - cmdstanpy - INFO - Chain [1] start processing
01:40:38 - cmdstanpy - INFO - Chain [1] done processing
01:40:39 - cmdstanpy - INFO - Chain [1] start processing
01:40:39 - cmdstanpy - INFO - Chain [1] done processing
01:40:39 - cmdstanpy - INFO - Chain [1] start processing
01:40:39 - cmdstanpy - INFO - Chain [1] done processing
01:40:39 - cmdstanpy - INFO - Chain [1] start processing
01:40:39 - cmdstanpy - INFO - Chain [1] done processing


------ Fold 4--------


01:40:39 - cmdstanpy - INFO - Chain [1] start processing
01:40:40 - cmdstanpy - INFO - Chain [1] done processing
01:40:40 - cmdstanpy - INFO - Chain [1] start processing
01:40:40 - cmdstanpy - INFO - Chain [1] done processing
01:40:40 - cmdstanpy - INFO - Chain [1] start processing
01:40:40 - cmdstanpy - INFO - Chain [1] done processing
01:40:40 - cmdstanpy - INFO - Chain [1] start processing
01:40:40 - cmdstanpy - INFO - Chain [1] done processing
01:40:41 - cmdstanpy - INFO - Chain [1] start processing


------ Fold 5--------


01:40:41 - cmdstanpy - INFO - Chain [1] done processing
01:40:41 - cmdstanpy - INFO - Chain [1] start processing
01:40:41 - cmdstanpy - INFO - Chain [1] done processing
01:40:41 - cmdstanpy - INFO - Chain [1] start processing
01:40:41 - cmdstanpy - INFO - Chain [1] done processing
01:40:41 - cmdstanpy - INFO - Chain [1] start processing
01:40:41 - cmdstanpy - INFO - Chain [1] done processing
01:40:42 - cmdstanpy - INFO - Chain [1] start processing
01:40:42 - cmdstanpy - INFO - Chain [1] done processing


In [20]:
# Result of validation set on each fold
final_result.sort_values(by="Fold",ascending=False)

,Brand,MAPE,Fold
24,orange-power,5.804694,5.0
23,lemon-boost,6.218594,5.0
22,kinder-cola,5.252834,5.0
21,gazoza,10.742014,5.0
20,adult-cola,5.304644,5.0
19,orange-power,7.972170,4.0
18,lemon-boost,6.218352,4.0
17,kinder-cola,5.481717,4.0
16,gazoza,13.349993,4.0
15,adult-cola,11.457783,4.0


## LSTM using darts

### Note on LSTM Model Performance and Nested CV

LSTM models typically require large training datasets for optimal performance. However, in our case, the use of nested cross-validation results in smaller training dataset sizes. As a consequence, the model performance may be affected negatively.

It's worth noting that for higher values of K (folds), LSTM may become impractical, as the minimum training data size required by LSTM is 25. Given our limited training data size, adjustments to the cross-validation strategy may be necessary to accommodate the specific requirements of LSTM models.


In [42]:
"""Given our limited training data size, increasing the number of folds beyond 1 in the cross-validation process
will further reduce the training dataset. This reduction may lead to an error with the LSTM model, as it requires a
minimum training data size (25)."""

#Reimporting the mape as the mape variable is used above
from darts.metrics import mape
cv = NestedCV(1)
splits = cv.split(resampled_data, "date")
for train, validate in splits:
    train_series_dict={}
    train.set_index("date",inplace=True)
    validate.set_index("date",inplace=True)
    for brand, group in train.groupby('brand'): 
        group['quantity'] = group['quantity'].fillna(method='ffill')
        ts= TimeSeries.from_dataframe(group[['quantity']], freq='M') 
        scaler_obj= Scaler() 
        ts=scaler_obj.fit_transform(ts)
        train_series_dict[brand]=ts
    all_series = list(train_series_dict.values())
    model = RNNModel(model='LSTM', input_chunk_length=4, output_chunk_length=1, n_epochs=5) 
    # model = NBEATSModel(input_chunk_length=3, output_chunk_length=1, n_epochs=5)
    # model = XGBModel(lags=10) 
    model.fit(all_series)
    for brand, group in validate.groupby('brand'):
        group['quantity'] = group['quantity'].fillna(method='ffill') 
        ts= TimeSeries.from_dataframe(group[['quantity']], freq='M') 
        forecast = model.predict(n=len(ts),series=train_series_dict[brand]) 
        # Evaluate performance using MAPE 
        forecast= scaler_obj.inverse_transform(forecast) 
        mape_value  = mape(ts, forecast)
        print(f"Mape of {brand} on validation set is : {mape_value}")
        

ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 2.8 K 
4 | V             | Linear           | 26    
---------------------------------------------------
2.8 K     Trainable params
0         Non-trainable params
2.8 K     Total params
0.011     Total estimated model params size (MB)


Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=5` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

Predicting: |                                                                                    | 0/? [00:00<…

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Mape of lemon-boost on validation set is : 28.34421775130288


Predicting: |                                                                                    | 0/? [00:00<…

Mape of orange-power on validation set is : 24.50222455712496
